Арцишевский Антон

In [1]:
import sqlite3
from html import unescape
from bs4 import BeautifulSoup
import re
import random
from fake_useragent import UserAgent
import time
import requests
from pprint import pprint

In [351]:
conn = sqlite3.connect('otaku_news.db')
cur = conn.cursor()

In [360]:
cur.execute("""
CREATE TABLE IF NOT EXISTS texts 
(id int PRIMARY KEY, date int, title text, category text)
""")

cur.execute("""
CREATE TABLE IF NOT EXISTS add_info
(id int PRIMARY KEY, tag_name text, likes int) 
""")

cur.execute("""
CREATE TABLE IF NOT EXISTS text_to_add_info
(id INTEGER PRIMARY KEY AUTOINCREMENT, id_text int, id_tag int) 
""")

conn.commit()
conn.close()

Попробую сначала по-порядку всё сделать

In [5]:
session = requests.session()
ua = UserAgent(verify_ssl=False)

In [140]:
page_number = 1
url = f'https://otakumode.com/news/anime?page={page_number}.html'
req = session.get(url, headers={'User-Agent': ua.random})
page = req.text
soup = BeautifulSoup(page, 'html.parser')

In [8]:
news = soup.find_all('h3', {'class': 'p-article__title'})
title = news[0].find('a').text
title

'Guide on Donating to Kyoto Animation'

In [9]:
attrs = news[0].find('a').attrs
attrs

{'class': ['inherit'],
 'href': '/news/5d4000fdad221889188b9e55/Guide-on-Donating-to-Kyoto-Animation',
 'onclick': "tom.helper.ga.push(['event', 'News', 'Clicked', 'CategoryFeatured_5d4000fdad221889188b9e55', 1]);"}

In [10]:
href = news[0].find('a').attrs['href']
href

'/news/5d4000fdad221889188b9e55/Guide-on-Donating-to-Kyoto-Animation'

Сделаю ссылку

In [11]:
url_one = 'https://otakumode.com'+href
url_one

'https://otakumode.com/news/5d4000fdad221889188b9e55/Guide-on-Donating-to-Kyoto-Animation'

In [18]:
req = session.get(url_one, headers={'User-Agent': ua.random})
page = req.text

soup = BeautifulSoup(page, 'html.parser')

### ТЕКСТ СТАТЬИ

In [90]:
full_text = soup.find_all('div', {'class': 'p-article__text'})
text = []
for i in full_text:
    i = str(i)
    paragraph = re.findall(r'<p>(.+?)</p>', i)
    paragraph = paragraph[0]
    paragraph = re.sub(r'<.*?>', '', paragraph)
    text.append(paragraph)
all_paragraphs = ''
for i in text:
    all_paragraphs += i
all_paragraphs

"Renowned anime studio Kyoto Animation suffered an arson attack on July 18th, 2019 (JST), which took over 30 lives and injured more. As the studio behind countless beloved anime like K-ON!, The Melancholy of Suzumiya Haruhi and Hibike! Euphonium, Kyoto Animation has received an outpouring of love from fans all around the world. Those who wish to support their recovery can do so by donating through the official channels below.*The information in this article is accurate as of August 1st, 2019. For future updates please check Kyoto Animation’s official website.”  Donating directly to Kyoto AnimationBANK NAME : THE KYOTO SHINKIN BANKSWIFT : KYSBJPJZBRANCH NAME : MINAMI MOMOYAMA BRANCHBRANCH NUMBER : 048ADDRESS : 16-50, YOSAI, MOMOYAMA-CHO, HUSHIMI-KU, KYOTO-SHI, KYOTO-HU, 612-8016 , JAPANACCOUNT NUMBER : 0002890ACCOUNT HOLDER : KYOTO ANIMATION CO.,LTD., REPRESENTATIVE DIRECTOR, HATTA HIDEAKI Donating through the Association of Japanese AnimatorsBANK NAME : MUFG Bank, LTD.SWIFT : BOTKJPJTB

In [129]:
meta = soup.find('a', {'class': 'c-btn c-btn--sm c-btn--icon-left c-btn--tag'})
meta = str(meta)

tags = re.findall(r'</i>(.*?)</a>', meta)
print(tags)

['Anime']


### А теперь по функциям

In [130]:
months = {
    value: key+1
    for key, value in enumerate(
        ['янв', 'фев', 'мар', 'апр', 'мая', 'июн', 'июл', 'авг', 'сен', 'окт', 'ноя', 'дек']
    )
}

In [235]:
n = 0
page_number = n
url = f'https://otakumode.com/news/anime?page={page_number}.html'
req = session.get(url, headers={'User-Agent': ua.random})
page = req.text
soup = BeautifulSoup(page, 'html.parser')
news = soup.find_all('h3', {'class': 'p-article__title'})

In [150]:
def parse_first_level_info(one_block):
    block = {}
    block['title'] = one_block.find('a').text
    block['href'] = one_block.find('a').attrs['href']
    return block

In [340]:
def parse_second_level_info(block):
    url_one = 'https://otakumode.com' + block['href']
    req = session.get(url_one, headers={'User-Agent': ua.random})
    page = req.text
    soup = BeautifulSoup(page, 'html.parser')
    
    dates = soup.find('time', {'class': 'p-article__time'})
    dates = str(dates)
    dates = re.search('00">(.+?)</time>', page)
    block['date'] = dates.group(1)
    
    full_text = soup.find_all('div', {'class': 'p-article__text'})
    text = []
    for i in full_text:
        i = str(i)
        paragraph = re.findall(r'<p>(.+?)</p>', i)
        paragraph = paragraph[0]
        paragraph = re.sub(r'<.*?>', '', paragraph)
        text.append(paragraph)
    all_paragraphs = ''
    for i in text:
        all_paragraphs += i
    block['full_text'] = all_paragraphs
    
    meta = soup.find_all('a', {'class': 'c-btn c-btn--sm c-btn--icon-left c-btn--tag'})
    meta = str(meta)
    tags = re.findall(r'</i>(.*?)</a>', meta)
    block['tags'] = tags
    return block

In [268]:
def get_nth_page(page_number):
    url = f'https://otakumode.com/news/anime?page={page_number}.html'
    req = session.get(url, headers={'User-Agent': ua.random})
    page = req.text
    soup = BeautifulSoup(page, 'html.parser')
    news = soup.find_all('h3', {'class': 'p-article__title'})
    blocks = []
    for n in news:
        try:
            blocks.append(parse_first_level_info(n))
        except Exception as e:
            print(e)
    result = []
    for b in blocks:
        result.append(b['href'])
    return result

In [366]:
### Не работает
def write_to_db(block):
    tags = []
    for tag in block['tags']:
        if tag in db_tags:
            tags.append(db_tags[tag])
        else:
            db_tags[tag] = len(db_tags) + 1 
            cur.execute('INSERT INTO add_info VALUES (?, ?)', (len(db_add_info), tag))
            conn.commit()
            tags.append(db_tags[tag])
    text_id = len(seen_news) + 1
    cur.execute(
        'INSERT INTO texts VALUES (?, ?, ?, ?)',
        (text_id,
         block['date'],
         block['title'], block['full_text'])
    )
    tags = [(text_id, t) for t in tags]
    cur.executemany(
        'INSERT INTO text_to_add_info (id_text, id_tag) VALUES (?, ?)',
        tags
    )
    conn.commit()
    seen_news.add(block['otaku_id'])

## Инфа с первой страницы со статьями

In [ ]:
conn = sqlite3.connect('otaku_news.db')
cur = conn.cursor()
cur.execute('SELECT tag_name, id FROM add_info')

db_tags = {}
for name, idx in cur.fetchall():
    print(name, idx)
    db_tags[name] = idx

In [367]:
n = 1
page_number = n
url = f'https://otakumode.com/news/anime?page={page_number}.html'
req = session.get(url, headers={'User-Agent': ua.random})
page = req.text
soup = BeautifulSoup(page, 'html.parser')
news = soup.find_all('h3', {'class': 'p-article__title'})

for block in news:
    blog1 = parse_first_level_info(block)
    titlelinktexttag = parse_second_level_info(blog1)

Ну я выкачал информацию про все статьи и внутреннию их информацию с первой страницы сайта. Остальные страницы я не сделал и в дб не запихнул